In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [10]:
# Read the CSV file
df=pd.read_csv('results_good7b.csv')

df.head()

,persona,source,text,Itemnum,answer
0,Barack Obama,NPP,Is talkative,1,5
1,Barack Obama,NPP,Tends to find fault with others,2,4
2,Barack Obama,NPP,Does a thorough job,3,5
3,Barack Obama,NPP,"Is depressed, blue",4,5
4,Barack Obama,NPP,"Is original, comes up with new ideas",5,4


In [11]:
df

,persona,source,text,Itemnum,answer
0,Barack Obama,NPP,Is talkative,1,5
1,Barack Obama,NPP,Tends to find fault with others,2,4
2,Barack Obama,NPP,Does a thorough job,3,5
3,Barack Obama,NPP,"Is depressed, blue",4,5
4,Barack Obama,NPP,"Is original, comes up with new ideas",5,4
...,...,...,...,...,...
18783,Gerhard Fischer,GPP,"Likes to reflect, play with ideas",40,3
18784,Gerhard Fischer,GPP,Has few artistic interests,41,3
18785,Gerhard Fischer,GPP,Likes to cooperate with others,42,5
18786,Gerhard Fischer,GPP,Is easily distracted,43,5


In [12]:

# Define the scoring keys
scoring_key = {
    'Extraversion': [1, 6, 11, 16, 21, 26, 31, 36],
    'Agreeableness': [2, 7, 12, 17, 22, 27, 32, 37, 42],
    'Conscientiousness': [3, 8, 13, 18, 23, 28, 33, 38, 43],
    'Neuroticism': [4, 9, 14, 19, 24, 29, 34, 39],
    'Openness': [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]
}

# Define reverse-scored items
reverse_scored = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]
reverse_scored = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]

# Function to adjust scores
def adjust_score(row):
    #print("for this row", row['persona'],"$$$$$", row['Itemnum'])
    if row['Itemnum'] in reverse_scored:
        #print("hit")
        return 6 - row['answer']
    return row['answer']

# Add adjusted score column
df['adjusted_score'] = df.apply(adjust_score,axis=1)
#print(df[:44])
# Function to calculate trait score
def calculate_trait_score(persona_data, trait_items):
    scores = []
    for item in trait_items:
        #print("§§§§§§§§§§§§§§§§§§§§")
        score = persona_data.loc[persona_data['Itemnum'] == item, 'answer'].iloc[0]
        if item in reverse_scored:
            score = 6 - score  # Reverse the score
        #print('§§§',score)    
        scores.append(score)
        
    return np.mean(scores)

# Group by persona and calculate scores
results = []
for persona, group in df.groupby('persona'):
    scores = {}
    for trait, items in scoring_key.items():
        scores[trait] = calculate_trait_score(group, items)
    results.append({'Persona': persona, **scores})

# Create a dataframe with the results
results_df = pd.DataFrame(results)

# Display the results
print(results_df)
#results_df.to_csv('bfi_scores.csv', index=False)

                    Persona  Extraversion  Agreeableness  Conscientiousness  \
0          A. T. Ariyaratne         2.625       3.333333           4.000000   
1         A.H.M. Noman Khan         3.875       3.333333           3.777778   
2             Abdon Nababan         4.000       2.555556           2.666667   
3       Abdul Razak Hussein         3.250       2.777778           3.333333   
4        Abdul Samad Ismail         3.125       4.000000           3.555556   
..                      ...           ...            ...                ...   
422         Zacarias Sarian         3.000       2.888889           4.000000   
423     Zafrullah Chowdhury         3.375       2.888889           2.888889   
424  Zakiah Hanum Abd Hamid         2.750       3.444444           4.000000   
425           Élie Ducommun         3.500       2.777778           2.777778   
426     Óscar Arias Sánchez         2.875       2.777778           3.888889   

     Neuroticism  Openness  
0          3.250      

In [15]:
results_df=results_df.round(3)
results_df.to_csv('BFI44-F7bI-G.csv', index=False)

In [16]:
import numpy as np
import pandas as pd

# Define the main function to process the data and calculate scores
def calculate_personality_scores(df, scoring_key, reverse_scored):
    # Function to adjust scores
    def adjust_score(row):
        if row['Itemnum'] in reverse_scored:
            return 6 - row['answer']
        return row['answer']

    # Add adjusted score column
    df['adjusted_score'] = df.apply(adjust_score, axis=1)
    
    # Function to calculate trait score for a single persona
    def calculate_trait_score(persona_data, trait_items):
        scores = []
        for item in trait_items:
            filtered_data = persona_data.loc[persona_data['Itemnum'] == item, 'answer']
            
            # Check if filtered_data is not empty
            if not filtered_data.empty:
                score = filtered_data.iloc[0]
                if item in reverse_scored:
                    score = 6 - score  # Reverse the score
                scores.append(score)
            else:
                print(f"Warning: Item {item} not found for persona {persona_data['persona'].iloc[0]}")
        
        # Return the mean of scores if scores list is not empty, else return None or np.nan
        return np.mean(scores) if scores else np.nan

    # Group by persona and calculate scores
    results = []
    for persona, group in df.groupby('persona'):
        scores = {}
        for trait, items in scoring_key.items():
            scores[trait] = calculate_trait_score(group, items)
        results.append({'Persona': persona, **scores})

    # Create a dataframe with the results
    results_df = pd.DataFrame(results)
    results_df = results_df.round(3)  # Round the results to 3 decimal places

    return results_df

# Example usage of the function:
# Define your scoring keys and reverse-scored items
scoring_key = {
    'Extraversion': [1, 6, 11, 16, 21, 26, 31, 36],
    'Agreeableness': [2, 7, 12, 17, 22, 27, 32, 37, 42],
    'Conscientiousness': [3, 8, 13, 18, 23, 28, 33, 38, 43],
    'Neuroticism': [4, 9, 14, 19, 24, 29, 34, 39],
    'Openness': [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]
}

reverse_scored = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]

# Assuming you have a DataFrame `df` defined elsewhere
results_df = calculate_personality_scores(df, scoring_key, reverse_scored)

# Print or save the results
print(results_df)
# results_df.to_csv('bfi_scores.csv', index=False)


                    Persona  Extraversion  Agreeableness  Conscientiousness  \
0          A. T. Ariyaratne         2.625          3.333              4.000   
1         A.H.M. Noman Khan         3.875          3.333              3.778   
2             Abdon Nababan         4.000          2.556              2.667   
3       Abdul Razak Hussein         3.250          2.778              3.333   
4        Abdul Samad Ismail         3.125          4.000              3.556   
..                      ...           ...            ...                ...   
422         Zacarias Sarian         3.000          2.889              4.000   
423     Zafrullah Chowdhury         3.375          2.889              2.889   
424  Zakiah Hanum Abd Hamid         2.750          3.444              4.000   
425           Élie Ducommun         3.500          2.778              2.778   
426     Óscar Arias Sánchez         2.875          2.778              3.889   

     Neuroticism  Openness  
0          3.250      

In [18]:
df2=pd.read_csv('results_bad7b.csv')
df2.tail()

,persona,source,text,Itemnum,answer
19663,Manuel Noriega,Dictator,"Likes to reflect, play with ideas",40,1
19664,Manuel Noriega,Dictator,Has few artistic interests,41,3
19665,Manuel Noriega,Dictator,Likes to cooperate with others,42,5
19666,Manuel Noriega,Dictator,Is easily distracted,43,5
19667,Manuel Noriega,Dictator,"Is sophisticated in art, music, or Literature",44,4


In [19]:
# Assuming you have a DataFrame `df` defined elsewhere
results_df2 = calculate_personality_scores(df2, scoring_key, reverse_scored)

# Print or save the results
print(results_df2)
# results_df.to_csv('bfi_scores.csv', index=False)


                     Persona  Extraversion  Agreeableness  Conscientiousness  \
0                  2 Pistols         3.375          3.000              3.000   
1         Abdelhamid Abaaoud         3.250          2.778              3.556   
2            Abdolmalek Rigi         3.000          3.667              2.333   
3         Abdulaziz al-Omari         2.500          2.444              3.111   
4    Abdullah Ahmed Abdullah         2.500          3.778              3.444   
..                       ...           ...            ...                ...   
442               Yevno Azef         2.875          3.000              3.000   
443           Yoo Young-chul         3.000          3.667              2.444   
444            Yoshio Kodama         3.500          2.556              3.444   
445            Zodiac Killer         2.125          3.667              2.889   
446  Ángel Maturino Reséndiz         2.875          3.333              3.333   

     Neuroticism  Openness  
0         

In [20]:
results_df2.to_csv('BFI44-F7bI-B.csv', index=False)

In [22]:
df3=pd.read_csv('results_neutral7b.csv')
df3.head()

,persona,source,text,Itemnum,answer
0,Michael Jackson,Actor,Is talkative,1,1
1,Michael Jackson,Actor,Tends to find fault with others,2,2
2,Michael Jackson,Actor,Does a thorough job,3,4
3,Michael Jackson,Actor,"Is depressed, blue",4,4
4,Michael Jackson,Actor,"Is original, comes up with new ideas",5,1


In [23]:
# Assuming you have a DataFrame `df` defined elsewhere
results_df3 = calculate_personality_scores(df3, scoring_key, reverse_scored)

# Print or save the results
print(results_df3)
# results_df.to_csv('bfi_scores.csv', index=False)
results_df3.to_csv('BFI44-F7bI-N.csv', index=False)

                    Persona  Extraversion  Agreeableness  Conscientiousness  \
0                   50 Cent         2.500          3.889              3.889   
1               A. A. Milne         2.875          3.111              3.111   
2    Abdelkader El Djezairi         2.625          3.333              3.111   
3                Ada Yonath         2.875          3.111              2.889   
4                  Agnez Mo         2.625          3.778              1.889   
..                      ...           ...            ...                ...   
435            Yury Luzhkov         3.000          3.556              2.778   
436             Zack Snyder         3.125          1.778              2.667   
437     Zbigniew Brzezinski         3.000          3.333              3.000   
438           Zhan Beleniuk         3.125          3.889              3.000   
439              Édith Piaf         3.500          3.444              2.444   

     Neuroticism  Openness  
0          3.000      